<a href="https://colab.research.google.com/github/jeraldkim/DataScienceProjects/blob/main/Scraping_Anime_from_myanimelist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import regex as re
import time

In [74]:
anime_list = []
num_anime = 200 #ex: 5 means 5*50 we want top 250 anime 0 would mean only top 50 anime so not run loop
n = 0

bug1 = 0
bug2 = 0

while n < num_anime:
    if n == 0:
      curr_link = "https://myanimelist.net/topanime.php"
    elif n > 0:
      curr_link = "https://myanimelist.net/topanime.php?limit=" + str(n * 50)
    n += 1

    response = requests.get(curr_link)
    html_content = response.content
    soup = BeautifulSoup(html_content, "html.parser")
    table = soup.find("table", class_="top-ranking-table")
    time.sleep(2)


    if table == None:
      print(curr_link)
      print(soup)
      bug1 = response
      bug2 = html_content
      bug3 = soup
    
    for row in table.find_all("tr")[1:51]:
      anime = {}
      anime["Rank"] = int(row.find("td", class_="rank").text)
      anime["Title"] = row.find("div", class_="di-ib clearfix").find("a").text
      anime["Link"] = row.find("a").get("href")
      anime["Score"] = float(row.find("td", class_="score").text)
      anime_list.append(anime)

anime_df = pd.DataFrame(anime_list)

In [75]:
anime_df = pd.DataFrame(anime_list)
anime_df

,Rank,Title,Link,Score
0,1,Fullmetal Alchemist: Brotherhood,https://myanimelist.net/anime/5114/Fullmetal_A...,9.11
1,2,Bleach: Sennen Kessen-hen,https://myanimelist.net/anime/41467/Bleach__Se...,9.10
2,3,Steins;Gate,https://myanimelist.net/anime/9253/Steins_Gate,9.08
3,4,GintamaÂ°,https://myanimelist.net/anime/28977/GintamaÂ°,9.07
4,5,Kaguya-sama wa Kokurasetai: Ultra Romantic,https://myanimelist.net/anime/43608/Kaguya-sam...,9.07
...,...,...,...,...
9995,9996,Ultra Nyan 2: The Great Happy Operation,https://myanimelist.net/anime/4682/Ultra_Nyan_...,5.89
9996,9997,Ushinawareta Choushoku,https://myanimelist.net/anime/40171/Ushinaware...,5.89
9997,9998,World Fool News (TV),https://myanimelist.net/anime/18205/World_Fool...,5.89
9998,9999,Xiongmao Monogatari TaoTao,https://myanimelist.net/anime/24603/Xiongmao_M...,5.89


In [76]:
from google.colab import drive

In [80]:
# drive.mount('/content/drive')
# path = '/content/drive/My Drive/output.csv'

# with open(path, 'w', encoding = 'utf-8-sig') as f:
#   anime_df.to_csv(f)

#method to save df to csv file on google drive

In [ ]:
new_anime_list = []

for link in anime_df["Link"]:

    response = requests.get(link)
    html_content = response.content
    soup = BeautifulSoup(html_content, "html.parser")
    
    anime = {}
    anime_type = soup.find("span",class_ ="dark_text",string="Type:")
    if anime_type == None:
      anime["Type"] = None
    else:
      anime["Type"] = anime_type.find_parent().find("a").text #Type of Show
    num_episodes = soup.find("span",class_ ="dark_text",string="Episodes:")
    if num_episodes == None:
      anime["# Episodes"] = None
    else:
      anime["# Episodes"] = num_episodes.find_parent().text #Number of Episodes
    
    source = soup.find("span",class_ ="dark_text",string="Source:")
    if source == None:
      anime["Source"] = None
    else:
      anime["Source"] = source.find_parent().text #Source
    
    aired_date = soup.find("span",class_ ="dark_text",string="Aired:")
    if aired_date == None:
      anime["Aired Date"] = None
    else:
      anime["Aired Date"] = aired_date.find_parent().text #Aired Date
    
    studio = soup.find("span",class_ ="dark_text",string="Studios:")
    if studio == None:
      anime["Studio"] = None
    else: 
      anime["Studio"] = studio.find_parent().text #Studio

    genre = soup.find("span",class_ ="dark_text",string=re.compile(r'Genres:|Genre:'))
    if genre == None:
      anime["Genre"] = None
    else:
      anime["Genre"] = genre.find_parent().text #Genre


    demo = soup.find("span",class_ ="dark_text",string="Demographic:")
    if demo == None:
      anime["Demographic"] = None
    else:
      anime["Demographic"] = demo.find_parent().text #Demographic
    
    duration = soup.find("span",class_ ="dark_text",string="Duration:")
    if duration == None:
      anime["Duration"] = None
    else:
      anime["Duration"] = duration.find_parent().text #Duration

    age_rating = soup.find("span",class_ ="dark_text",string="Rating:")
    if age_rating == None:
      anime["Age Rating"] = None
    else:
      anime["Age Rating"] = age_rating.find_parent().text #Age Rating
      
    new_anime_list.append(anime)    

new_anime_df = pd.DataFrame(new_anime_list)

In [ ]:
new_anime_list

In [ ]:
test_df = pd.DataFrame(new_anime_list)
test_df

In [ ]:
test_df[test_df['Type'] == None]